In [ ]:
from datetime import datetime
import re

symbolRegex="[:][0-9]+"
ISINRegex="[:]\s[A-Z0-9]+\s[a-zA-Z]+"


def findSymbol(target):
    symbolRegex="[:][0-9]+"
    return re.findall(symbolRegex,target)

def findISIN(target):
    ISINRegex="[:]\s[A-Z0-9]+\s[a-zA-Z]+"
    return re.findall(ISINRegex,target)

def findBuySell(arr):
    regex="[0-9]*\s[@]\s[0-9]+[.][0-9]*\s[=]\s[0-9]*[.][0-9]*\s[a-zA-Z]*[:]\s[0-9]*[.][0-9]*"
    return [i for i in arr if re.findall(regex,i)]

def isData(arr):
    if (len(arr)<4):
        return False
    res=False
    try:
        datetime.strptime(arr[1],"%H:%M:%S")
        res=True
    except ValueError:
        return False
    return arr[0].isdigit() and len(arr[0])>=8 and arr[1] and res


In [ ]:
import pdfplumber
import pandas as pd

pdf,pdf2='example.pdf','example2.pdf'

columns=['Order No.','OrderTime','TradeNo.','TradeTime',
'Security/ContractDescription','Buy(B) /Sell(S)','Quantity','Gross Rate/trade price per Unit(Rs)','Brokerage perUnit','NetRate perUnit(Rs)'
,'NetTotal(BeforeLevies)(Rs.)','Symbol','ISIN','Total Buy','Total Sell'
]

def removeNone(array):
   return [v for v in array if v is not None and v is not '']

def createOrganizedDf(page):
    row=[]
    temp=[]
    for elem in page:
        ISIN=findISIN(elem[0])
        Symbol=findSymbol(elem[0])
        buySell=findBuySell(elem)
        if len(Symbol)==1:
            temp+=Symbol
        if len(ISIN)==1:
            temp+=ISIN
        elif len(buySell)>0:
            if (temp[5]=="B"):   
                temp+=[buySell[0],None]
            else:
                temp+=[None,buySell[0]]
        elif isData(elem):
            temp=elem+temp
        if len(temp)==15:
            row.append(temp)
            temp=[]
    df=pd.DataFrame(row,columns=columns)
    return df



pdf=pdfplumber.open(pdf)
pages=[]
for i in pdf.pages:
    pages+=i.extract_table(table_settings={'keep_blank_chars':False})

# remove the headers and useless data
del pages[0][0:2]

# remove all useless values like none,null,or " "
for i in range(len(pages)):
    pages[i]=removeNone(pages[i])
df=createOrganizedDf(pages)
df

